## Imports

In [ ]:
from datetime import datetime as dt
from pathlib import Path
from hbt_tools import utils_hbt as uh
from matplotlib import rcParams
from matplotlib import dates as mdates
import matplotlib.pyplot as plt

## Settings

In [ ]:
# Paths
path_raindata = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Regendaten\Cham Niederschlag"
    r"\CHZ_Niederschlagsumme.csv")
save_path = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Plots")

In [ ]:
# Installation: 21.04.2022 10:00
# Demontierung: 29.08.2022
dt_format = '%d.%m.%Y %H:%M'
EZG_tot = 5.18 # ha
rcParams.update({'figure.autolayout': True})
dt_start = dt.strptime('07.06.2022 00:00', dt_format)
dt_end = dt.strptime('08.06.2022 00:00', dt_format)
if 'dt_end' not in vars():
    dt_end = dt.now()

In [ ]:
# Events
events = {
    'Installation Messtechnik': dt(2022, 4, 21),
    'Probenahme 1': dt(2022, 6, 1),
    'Probenahme 2': dt(2022, 7, 4),
    'Probenahme 3': dt(2022, 8, 29),
}

## Read Data

In [ ]:
df_rain = uh.read_raindata_meteoschweiz(path_raindata)

## Modify Data

In [ ]:
mask = ((df_rain.index>=events['Installation Messtechnik']) &
    (df_rain.index<=events['Probenahme 3']))
df_rain = df_rain[mask]
df_rain['kumulativ [mm]'] = df_rain.cumsum()

## Plot Data

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(df_rain.index, df_rain.iloc[:,1], 'b--')
ax1.axvspan(events['Installation Messtechnik'], events['Probenahme 1'],
    color='red', alpha=0.5)
ax1.axvspan(events['Probenahme 1'], events['Probenahme 2'],
    color='grey', alpha=0.5)
ax1.axvspan(events['Probenahme 2'], events['Probenahme 3'],
    color='red', alpha=0.5)

for k, v in events.items():
    ax1.text(x=v, y=440, s=k, rotation=15)
    ax1.annotate(text="", xy=(v, 350), xytext=(v, 440),
        arrowprops=dict(facecolor='black', shrink=0.005, width=1, 
        headwidth=5))

ax2 = ax1.twinx()
ax2.plot(df_rain.index, df_rain.iloc[:,0], 'b')

ax1.set_xticklabels(ax1.get_xticks(), rotation=30)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y'))

ax1.set_xlabel('Datum')
ax1.set_ylabel('Regensumme [mm]')
ax2.set_ylabel('Regen [mm/10min]')

cm_in = 0.394
fig.set_size_inches(16*cm_in,8*cm_in)